In [22]:
# Install necessary libraries
!pip install transformers numpy pandas torch scikit-learn

# Import libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [23]:
# Set seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [24]:
# Load the fine-tuned BERT model
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [25]:
tokens = tokenizer.encode('pepsi is the best clapping hands fire', return_tensors='pt')
result = fine_tuned_model(tokens)
result

SequenceClassifierOutput(loss=None, logits=tensor([[-1.8174, -2.0136, -0.9212,  1.1183,  3.0924]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
# from google.colab import files
# files.upload()

In [27]:
df = pd.read_csv('Instagram Data.csv')

Cleaning the data, Converting emojis to words, and removing special characters

In [28]:
!pip install emoji
import re
from emoji import demojize

df['Comment Text'] = df['Comment Text'].str.lower().apply(demojize)
df['Comment Text'] = df['Comment Text'].apply(lambda x: re.sub(r'[:,@_\W]', ' ', x))

In [29]:
df

,Comment Text
0,anitta
1,pepsi is the best clapping hands fire
2,city
3,manchester city
4,manchester city fire fire soccer ball socc...
5,love it smiling face with heart eyes
6,top fire fire fire fire
7,let s go let s produce
8,i imagine a gourmet cart with all the informat...
9,face savoring food red circle blue circle ...


In [30]:
# Tokenize and preprocess your comments for sentiment analysis
encoded_data = tokenizer(df['Comment Text'].tolist(), padding=True, truncation=True, return_tensors='pt', max_length=128)

In [31]:
# Extract BERT embeddings for your comments
with torch.no_grad():
    outputs = fine_tuned_model(**encoded_data)
    logits = outputs.logits  # Use logits instead of pooler_output
    predicted_labels = torch.argmax(logits, dim=1) + 1  # Adding 1 because the model predicts labels from 0 to 4


# Map numeric sentiment labels to categories (Negative, Neutral, Positive)
sentiment_mapping = {1: 'Negative', 2: 'Negative', 3: 'Neutral', 4: 'Positive', 5: 'Positive'}
df['Sentiment_Class'] = pd.Series(predicted_labels.numpy()).map(sentiment_mapping)


KNN Classification

In [32]:
# Specify the feature (X) and target (y) variables for training
# The least populated class in y has only 3 members, was less than n_splits=5 so we adjusted the value of n_splits to 3.

X = logits.numpy()  # Assuming logits contains the BERT embeddings

# Use these predicted sentiment labels as target (y) for KNN
y = df['Sentiment_Class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {'n_neighbors': [3, 5, 7, 9, 11]}

knn_classifier = KNeighborsClassifier()

grid_search = GridSearchCV(knn_classifier, param_grid, cv=3)

grid_search.fit(X_train, y_train)

best_k = grid_search.best_params_['n_neighbors']

print("Best value for k:", best_k)

# Create the KNN classifier with the optimal k value
knn_classifier = KNeighborsClassifier(n_neighbors=best_k)

# Train the KNN classifier on the training data
knn_classifier.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = knn_classifier.predict(X_test)

# Evaluate the performance of the KNN model
print(classification_report(y_test, y_pred))


Best value for k: 3
              precision    recall  f1-score   support

    Positive       1.00      1.00      1.00        10

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10

